# Getting Mexico's schools and their careers

The objective of this repository is to get the data of Mexico's schools and their careers from an API and then saving it to a database using Entity Framework Core.

## Calling the API

First we are going to call the API. The data source is [DataMéxico](https://datamexico.org/en). I got the API link after playing a bit with their [Tesseract UI](https://api.datamexico.org/ui/), I'm only concerned about the careers that every institution offers, as well as the academic degree of every career.

In [ ]:
using System.Net.Http;

var client = new HttpClient();
var response = await client.GetAsync("https://api.datamexico.org/tesseract/data.jsonrecords?cube=anuies_enrollment&debug=true&drilldowns=Academic+Degree%2CCareer%2CInstitution&locale=es&measures=Students&parents=false&sparse=true");
var rawContent = await response.Content.ReadAsStringAsync();

If you open and inspect the API GET URL, you'll see that the response is a list of JSONs:

```json
{
    "data" : [
        {
            "Academic Degree ID": 8,
            "Academic Degree": "Estudios Técnicos o Comerciales con Preparatoria Terminada",
            "Career ID": 11200001,
            "Career": "Tecnico superior universitario en gestion de educacion y capacitacion",
            "Institution ID": 3345,
            "Institution": "Instituto Consorcio Clavijero",
            "Students": 13
        },
        ...
    ]
}
```

So let's generate a C# class to deserialize the response into a typed list:

In [ ]:
// Root myDeserializedClass = JsonConvert.DeserializeObject<Root>(myJsonResponse);
#r "nuget:Newtonsoft.Json"

using Newtonsoft.Json;

public class ResponseObject
{
    [JsonProperty("Academic Degree ID")]
    public int AcademicDegreeID { get; set; }

    [JsonProperty("Academic Degree")]
    public string AcademicDegree { get; set; }

    [JsonProperty("Career ID")]
    public int CareerID { get; set; }
    public string Career { get; set; }

    [JsonProperty("Institution ID")]
    public int InstitutionID { get; set; }
    public string Institution { get; set; }
    public int Students { get; set; }
}

## Parsing the API response

Now we know how to access the list and the names of the properties:

In [ ]:
#r "nuget:Newtonsoft.Json"

using Newtonsoft.Json.Linq;

var objects = JObject.Parse(rawContent)["data"];

In [ ]:
objects.First().ToString()

{
  "Academic Degree ID": 8,
  "Academic Degree": "Estudios Técnicos o Comerciales con Preparatoria Terminada",
  "Career ID": 11000059,
  "Career": "Licencia profesional en ciencias de la educacion",
  "Institution ID": 2965,
  "Institution": "Universidad Unilider S.C.",
  "Students": 6
}

In [ ]:
objects.First()["Career"].ToString()

Licencia profesional en ciencias de la educacion

## Referencing models and database context

Before processing the data to organize it as needed, we first need to import the class library project *Database*.
I used Entity Framework Core code first approach to map the objects to the database, so the DLL output of the project already contains the models.

Inside the project folder, add a `.connection` file and put your connection string there.
See [`example.connection`](/Database/example.connection).

The `SchoolsContext` class configures different table and property names for the models, according to my specific need.
Personally I would prefer to stick with the same class and property names, so the `OnModelCreating` method would be simpler.

Follow [these instructions](https://docs.microsoft.com/en-us/ef/core/managing-schemas/migrations/?tabs=dotnet-core-cli) to migrate the code to your database.
By doing this the project builds so the DLL will be already available.

In [ ]:
#r ".\Database\bin\Debug\net5.0\Database.dll"
#r "nuget:Microsoft.EntityFrameworkCore"

using Database;

## Processing and organizing the data

Let's start by filtering unique records of institutions, degrees, and careers:

In [ ]:
var institutions = objects
    .GroupBy(o => o["Institution ID"])
    .Select(g => g.First())
    .Select(o => new Institution {
        Name = o["Institution"].ToString(),
        OriginalId = (long)o["Institution ID"]
    })
    .OrderBy(i => i.Name)
    .ToList();

institutions

index,Id,Name,OriginalId,Careers,InstitutionCareers
0,0,"17, Instituto De Estudios Críticos",510,<null>,<null>
1,0,1972 Año De Juárez,3577,<null>,<null>
2,0,Academia De Arte Culinaria Internacional,178,<null>,<null>
3,0,Academia De Arte De Florencia,509,<null>,<null>
4,0,Academia De Arte Y Diseño,895,<null>,<null>
5,0,Academia De Ciencias Sociales Y Desarrollo Humano Acanits,3426,<null>,<null>
6,0,Academia De Estudios Superiores Constantino,2724,<null>,<null>
7,0,Academia De Integración Jurídica De Oaxaca,2341,<null>,<null>
8,0,Academia De Música Fermatta,3598,<null>,<null>
9,0,Academia De Música Solfeggio,1438,<null>,<null>


In [ ]:
var degrees = objects
    .GroupBy(o => o["Academic Degree ID"])
    .Select(g => g.First())
    .Select(o => new Degree {
        Name = o["Academic Degree"].ToString(),
        OriginalId = (long)o["Academic Degree ID"]
    })
    .OrderBy(d => d.Name)
    .ToList();

degrees

index,Id,Name,OriginalId,Careers
0,0,Doctorado,14,<null>
1,0,Especialidad,12,<null>
2,0,Estudios Técnicos o Comerciales con Preparatoria Terminada,8,<null>
3,0,Licenciatura,11,<null>
4,0,Maestría,13,<null>
5,0,Normal de Licenciatura,10,<null>


In [ ]:
var careers = objects
    .GroupBy(o => o["Career ID"])
    .Select(g => g.First())
    .Select(o => new Career {
        Name = o["Career"].ToString(),
        OriginalId = (long)o["Career ID"],
        DegreeId = (int)o["Academic Degree ID"]
    })
    .OrderBy(c => c.Name)
    .ToList();

careers

index,Id,Name,OriginalId,DegreeId,Degree,Institutions,InstitutionCareers
0,0,Abogado,33100537,11,<null>,<null>,<null>
1,0,"Abogado, notario y actuario",33100536,11,<null>,<null>,<null>
2,0,Actuario,53200029,11,<null>,<null>,<null>
3,0,Alta especialidad en anestesia de alto riesgo y medicina perioperatoria,91301011,12,<null>,<null>,<null>
4,0,Alta especialidad en broncoscopia y neumologia intervencionista,91332008,12,<null>,<null>,<null>
5,0,Alta especialidad en glaucoma,91335008,12,<null>,<null>,<null>
6,0,"Alta especialidad en minima invasion, endourologia y cirugia robotica en urologia",91344007,12,<null>,<null>,<null>
7,0,Alta especialidad en oculoplastica,91309005,12,<null>,<null>,<null>
8,0,Alta especialidad en retina y vitrio,91335007,12,<null>,<null>,<null>
9,0,Arquitecto,73100186,11,<null>,<null>,<null>
